This notebook is used to show:
1. How to save and reload the Flexcode model so not to have to re-train it every time;
2. How to select the best bump removal parameter from an array of potential value

In [1]:
import flexcode
import numpy as np
import xgboost as xgb
from flexcode.regression_models import XGBoost, CustomModel

from matplotlib import pyplot as plt
%matplotlib inline

## Data Creation

In [2]:
def generate_data(n_draws):
    x = np.random.normal(0, 1, n_draws)
    z = np.random.normal(x, 1, n_draws)
    return x, z

x_train, z_train = generate_data(1000)
x_validation, z_validation = generate_data(1000)
x_test, z_test = generate_data(1000)

## Saving and Reload Model

In [3]:
# Parameterize model
model = flexcode.FlexCodeModel(XGBoost, max_basis=31, basis_system="cosine",
                             regression_params={'max_depth': 3, 'learning_rate': 0.5, 'objective': 'reg:linear'})

# Fit model and predict on test data
model.fit(x_train, z_train)
cdes_predict_xgb, z_grid = model.predict(x_test, n_grid=200)

# Show output some general values of the first two predictions
# for further check
print(np.max(cdes_predict_xgb[7, :]))
print(np.max(cdes_predict_xgb[42, :]))

1.3204763840926326
1.315065820028842


In [4]:
# Print model parameters
model.__dict__

{'basis_system': 'cosine',
 'best_basis': range(0, 31),
 'bump_threshold': None,
 'max_basis': 31,
 'model': <flexcode.regression_models.XGBoost at 0x1173e35f8>,
 'sharpen_alpha': None,
 'z_max': array([4.98535658]),
 'z_min': array([-4.84002112])}

In [5]:
import pickle

# Saving the model
pickle.dump(file=open('flexcode_model.pkl', 'wb'), obj=model, 
            protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
# Reaload the model
model_reloaded = pickle.load(open('flexcode_model.pkl', 'rb'))

# Predict again
cdes_predict_xgb_reloaded, z_grid = model_reloaded.predict(x_test, n_grid=200)

In [7]:
# Same output as above
print(np.max(cdes_predict_xgb_reloaded[7, :]))
print(np.max(cdes_predict_xgb_reloaded[42, :]))

1.3204763840926326
1.315065820028842


In [8]:
# Print parameters of the reloaded model
model_reloaded.__dict__

{'basis_system': 'cosine',
 'best_basis': range(0, 31),
 'bump_threshold': None,
 'max_basis': 31,
 'model': <flexcode.regression_models.XGBoost at 0x1173d2b38>,
 'sharpen_alpha': None,
 'z_max': array([4.98535658]),
 'z_min': array([-4.84002112])}

## Spurious Bump Removal - Tune Using Validation Data

In [9]:
# Define a grid of values to tune over
bump_removal_grid = np.linspace(0.01, 0.2, 20)
print(bump_removal_grid)

[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13 0.14
 0.15 0.16 0.17 0.18 0.19 0.2 ]


In [10]:
# Parameterize model
model = flexcode.FlexCodeModel(XGBoost, max_basis=31, basis_system="cosine",
                             regression_params={'max_depth': 3, 'learning_rate': 0.5, 'objective': 'reg:linear'})

# Fit model
model.fit(x_train, z_train)

In [11]:
# Tune the model - the bump removal grid is passed directly 
# in the tune function

model.tune(x_validation, z_validation, 
           bump_threshold_grid=bump_removal_grid)

Under the hood, it selects the bump value corresponding to the smallest CDE loss on the validation data.

The best value is accessible among the attributes of the model, as below:

In [12]:
model.bump_threshold

0.01